In [2]:
import boto3
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from skimage.segmentation import slic
from skimage.color import label2rgb
from PIL import Image
import io
import json

# Set up S3 client
s3 = boto3.client('s3')
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(4, activation='softmax')  # Assuming 4 classes
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Train the model (this is a placeholder, you'd actually need to fit the model to your data)
# model.fit(...)

# Save the model
model.save('dental_classification_model.h5')

# Load the pre-trained classification model
model = load_model('dental_classification_model.h5')

# Function to download an image from S3
def download_from_s3(bucket, key):
    response = s3.get_object(Bucket=bucket, Key=key)
    image_bytes = response['Body'].read()
    return Image.open(io.BytesIO(image_bytes))
# Function to segment the image
def segment_image(image, num_segments=100):
    # Convert PIL Image to numpy array
    image_array = np.array(image)
    
    # Perform SLIC segmentation
    segments = slic(image_array, n_segments=num_segments, compactness=10)
    
    # Create a new image where each segment is filled with its average color
    segmented_image = label2rgb(segments, image_array, kind='avg')
    
    return Image.fromarray((segmented_image * 255).astype(np.uint8))

# Function to classify a single image
def classify_image(image):
    # Resize image to match model's expected input
    image = image.resize((224, 224))
    # Convert to array and preprocess
    img_array = tf.keras.preprocessing.image.img_to_array(image)
    img_array = tf.expand_dims(img_array, 0)  # Create batch axis
    img_array = img_array / 255.0  # Normalize
    
    # Make prediction
    predictions = model.predict(img_array)
    
    # Assuming we have these classes
    class_names = ['healthy', 'cavity', 'gum_disease', 'other']
    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = np.max(predictions[0])
    
    return predicted_class, confidence
# Main function to process images from S3
def process_s3_images(bucket_name, prefix):
    # List objects in the S3 bucket
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
    
    for obj in response['Contents']:
        key = obj['Key']
        if key.lower().endswith(('.png', '.jpg', '.jpeg')):
            print(f"Processing {key}")
            
            # Download image from S3
            image = download_from_s3(bucket_name, key)
            
            # Segment the image
            segmented_image = segment_image(image)
            
            # Classify the segmented image
            predicted_class, confidence = classify_image(segmented_image)
            print(f"Image: {key}")
            print(f"Predicted class: {predicted_class}")
            print(f"Confidence: {confidence:.2f}")
            print("---")
            
            # Optionally, you can save the results back to S3
            result = {
                'image': key,
                'predicted_class': predicted_class,
                'confidence': float(confidence)
            }
            s3.put_object(
                Bucket=bucket_name,
                Key=f"results/{key.split('/')[-1]}.json",
                Body=json.dumps(result)
            )
# Usage
bucket_name = 'sagemaker-studio-010526272250-hsf94lgtf6i'
prefix = 'Test_Dentex_Images/Panoramic_Dental_Xray_Dataset/'
process_s3_images(bucket_name, prefix)

Processing Test_Dentex_Images/Panoramic_Dental_Xray_Dataset/1.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
Image: Test_Dentex_Images/Panoramic_Dental_Xray_Dataset/1.jpg
Predicted class: gum_disease
Confidence: 0.28
---
Processing Test_Dentex_Images/Panoramic_Dental_Xray_Dataset/10.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Image: Test_Dentex_Images/Panoramic_Dental_Xray_Dataset/10.jpg
Predicted class: gum_disease
Confidence: 0.28
---
Processing Test_Dentex_Images/Panoramic_Dental_Xray_Dataset/100.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Image: Test_Dentex_Images/Panoramic_Dental_Xray_Dataset/100.jpg
Predicted class: gum_disease
Confidence: 0.27
---
Processing Test_Dentex_Images/Panoramic_Dental_Xray_Dataset/102.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Image: Test_Dentex_Images/Panoramic_Dental_Xray_Dataset/102.jpg
Predicted class: gum_disease
Confidence: 0.28
---
Processing Test_Dentex_Images/Panoramic_Dental_Xray_Dataset/103.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Image: Test_Dentex_